In [ ]:
import json

with open("alex_commands.json", "r", encoding="utf-8") as f:
    data = json.load(f)

training_data = []

# Check if JSON is a dict (with "keywords") or a list of objects
if isinstance(data, dict):
    for intent, details in data.items():
        for phrase in details.get("keywords", []):
            training_data.append({
                "text": phrase,
                "intent": intent
            })
elif isinstance(data, list):
    for item in data:
        if "text" in item and "intent" in item:
            training_data.append({
                "text": item["text"],
                "intent": item["intent"]
            })

with open("intent_training.json", "w", encoding="utf-8") as f:
    json.dump(training_data, f, indent=2, ensure_ascii=False)

print("Training dataset created!")


In [ ]:
import json
import pickle
import warnings
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score

# ----------------------------
# Load training data
# ----------------------------
with open("intent_training.json", "r", encoding="utf-8") as f:
    data = json.load(f)

texts = [item["text"] for item in data]
labels = [item["intent"] for item in data]
responses = [item.get("response", "") for item in data]  # optional response

# ----------------------------
# Convert text to vectors
# ----------------------------
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

# ----------------------------
# Train classifier
# ----------------------------
model = LogisticRegression(max_iter=2000)
model.fit(X, labels)

# ----------------------------
# Evaluate classifier
# ----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42, stratify=labels
)

y_pred = model.predict(X_test)

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average="weighted")
print(f"Accuracy: {accuracy:.4f}")
print(f"Weighted F1 Score: {f1:.4f}\n")
print("Classification Report:")
print(classification_report(y_test, y_pred, zero_division=0))

cv_scores = cross_val_score(model, X, labels, cv=5, scoring="f1_weighted")
print(f"Cross-validated F1 (weighted): {np.mean(cv_scores):.4f}")

# ----------------------------
# Save model, vectorizer, and responses
# ----------------------------
with open("intent_model.pkl", "wb") as f:
    pickle.dump({
        "model": model,
        "vectorizer": vectorizer,
        "responses": {item["intent"]: item.get("response", "") for item in data}
    }, f)

print("\n✅ Intent classifier trained and saved with responses as 'intent_model.pkl'")
